Set select and compile apk files for 55 experiment.

Goal: use 5 sample apps from google samples as well as 5 fdroid apps.

Deref random sampling done with next notebook `55SetupAndPushRemoteRun.ipynb`

In [1]:
val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"

interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._



path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._, Plotly._

import ujson.Value

import sys.process._



In [2]:
// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
// TODO: configure thread pool: https://docs.scala-lang.org/overviews/parallel-collections/configuration.html

import $ivy.$                                         

import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.Action

import edu.colorado.plv.bounder.RunConfig

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
// TODO: configure thread pool: https://docs.scala-lang.org/overviews/parallel-collections/configuration.html

Parse list of apps with APK files in the fdroid download folder
------------------------------------------------------------------------

Full data download: https://drive.google.com/file/d/1edtMA9Bt0g9rPf1gp34W1a9e-oUdINJd/view?usp=sharing

In [4]:
import scala.collection.parallel._
val fdroidBase = File("/Users/shawnmeier/Documents/data/fdroid")
class DataSet(rootFile:File){
    val rootSize = rootFile.toString.split("/").size
    val allApps = rootFile.list.toSet
    val appPackages = allApps.map(v => v.toString.split("/").last).toSet
    val apkFilesPar = rootFile.glob("**/*.apk").zipWithIndex.toList.par
    val forkJoinPool = new java.util.concurrent.ForkJoinPool(8)
    apkFilesPar.tasksupport = new ForkJoinTaskSupport(forkJoinPool)
    val apkFiles = apkFilesPar.flatMap{a => 
            // Slow - filters out proguard apps
            val isProguard = Driver.detectProguard(a._1.toString)
            println(s"index: ${a._2} apk: ${a._1} proguard: ${isProguard}")
//             val isProguard = false
            if(isProguard) None else Some(a._1)
        }.seq.toList
    val apkGroups = 
        apkFiles.groupBy(f => appPackages.find(ad => 
                                               f.toString.split("/")(rootSize) == ad.toString)
                         .getOrElse(throw new IllegalStateException(f.toString)))

    override def toString():String = {
        "\n" + 
        s"number of app directories: ${allApps.size}\n" +
        s"number of distinct packages: ${appPackages.size}\n" +
        s"number of apk files: ${apkFiles.size}\n" +
        s"distinct apps with apks: ${apkGroups.size}\n"
    }
    /**
    * Return a list of apps that contain APK files
    */
    def getApps():List[String] = apkGroups.keySet.toList
    def apksForApp(app:String):List[File] = apkGroups(app)
    def getAppsWithNoApk():List[String] = 
        appPackages.filter(a => !apkGroups.contains(a.toString)).toList
    def nRandomApps(n:Integer):List[String] = {
        Random.setSeed(23L)
        val sh = Random.shuffle(apkGroups.keySet.toList)
        sh.take(n).toList
    }
}
// Note: this takes really long to run, partial list of outputs in fdroidBase / apkWithProguard.txt
//val fdroidData = new DataSet(fdroidBase)

import scala.collection.parallel._

fdroidBase: File = /Users/shawnmeier/Documents/data/fdroid
defined class DataSet

In [9]:
// Read list of apps and parse if proguarded (raw output from earlier cell)
val pgFile = fdroidBase / "apkWithProguard.txt"
val nonpgapks = pgFile.contentAsString.split("\n").flatMap{l =>
  val spl = l.split(" apk: ")(1).split(" proguard: ")
  val path = spl(0)
  if(spl(1).toBoolean) None else Some(path)
}

pgFile: File = /Users/shawnmeier/Documents/data/fdroid/apkWithProguard.txt
nonpgapks: Array[String] = Array(
  "/Users/shawnmeier/Documents/data/fdroid/org.bc_bd.mrwhite/1.1/apk/org.bc_bd.mrwhite_2.apk",
  "/Users/shawnmeier/Documents/data/fdroid/com.kodarkooperativet.notificationstopwatch/1.4/apk/com.kodarkooperativet.notificationstopwatch_6.apk",
  "/Users/shawnmeier/Documents/data/fdroid/me.shrimadhavuk.watransmitter/2.1/apk/me.shrimadhavuk.watransmitter_3.apk",
  "/Users/shawnmeier/Documents/data/fdroid/de.blocklink.pigrid/1.4/apk/de.blocklink.pigrid_5.apk",
  "/Users/shawnmeier/Documents/data/fdroid/com.github.characterdog.share_my_number/1.3/apk/com.github.characterdog.share_my_number_3.apk",
  "/Users/shawnmeier/Documents/data/fdroid/org.ligi.etheremote/0.2/apk/org.ligi.etheremote_2.apk",
  "/Users/shawnmeier/Documents/data/fdroid/io.homeassistant.android/1.0-fdroid/apk/io.homeassistant.android_4.apk",
  "/Users/shawnmeier/Documents/data/fdroid/org.t2.synconwifi/1.0.2/apk/org.t2

In [22]:
object ParseMeta{
    //import ujson._
    def getTargetVersion(path:String):Int = {
        val metaF = File(path.split("/apk/")(0)) / "meta.txt"
        ujson.read(metaF.contentAsString)("minSdkVersion") match {
            case ujson.Str(n) => n.toInt
            case ujson.Num(v) => v.toInt
        }
    }
}

val apkBySDK = nonpgapks.groupBy(ParseMeta.getTargetVersion).toList.sortBy(_._1).reverse
val apkOver20 = apkBySDK.filter(_._1 > 20).flatMap(_._2)

(fdroidBase / "apkOver20.txt").overwrite(apkOver20.mkString("\n"))

defined object ParseMeta
apkBySDK: List[(Int, Array[String])] = List(
  (
    24,
    Array(
      "/Users/shawnmeier/Documents/data/fdroid/com.mattermost.rnbeta/1.19.0/apk/com.mattermost.rnbeta_193.apk",
      "/Users/shawnmeier/Documents/data/fdroid/foundation.e.blisslauncher/1.2.1/apk/foundation.e.blisslauncher_12100.apk",
      "/Users/shawnmeier/Documents/data/fdroid/org.andstatus.app/51.02/apk/org.andstatus.app_298.apk",
      "/Users/shawnmeier/Documents/data/fdroid/de.sensebox.blockly/1.1.2/apk/de.sensebox.blockly_10102.apk"
    )
  ),
  (
    23,
    Array(
      "/Users/shawnmeier/Documents/data/fdroid/com.juliansparber.captiveportallogin/0.2/apk/com.juliansparber.captiveportallogin_2.apk",
      "/Users/shawnmeier/Documents/data/fdroid/org.elijaxapps.androidxmrigminer/1.02/apk/org.elijaxapps.androidxmrigminer_5.apk",
      "/Users/shawnmeier/Documents/data/fdroid/org.ninthfloor.copperpdf/1.2.1/apk/org.ninthfloor.copperpdf_4.apk",
      "/Users/shawnmeier/Documents/data/fdroi

In [10]:
// Randomly sample a set of apps with defined APK files
// Not currently used, manually grabbed 5 non-proguard apps
val expDir = File(
    "/Users/shawnmeier/Documents/source/bounder/experiments/5fdroid5dev_sampderef")

def pkgAndBaseToApk(pkg:String, base:File):File = {
    val apkFiles = (fdroidBase / pkg).glob("**/*.apk").toList
    assert(apkFiles.size == 1, s"apk for ${pkg} not found")
    apkFiles.head
}
def writeSample() = {
    val sample = fdroidData.nRandomApps(20).flatMap{ s => 
            val apkPath = pkgAndBaseToApk(s, fdroidBase).toString
            if(!Driver.detectProguard(apkPath)){Some(apkPath + ":" + s)} else None
        }.take(10)
    val experimentDir = expDir
    val appSample: File = experimentDir / "appSample.txt"
    println(appSample)
    appSample.createFile().append(sample.mkString("\n"))
}
//writeSample() // uncomment to run

expDir: File = /Users/shawnmeier/Documents/source/bounder/experiments/allDeref_fdroid_subset_15
defined function pkgAndBaseToApk
defined function writeSample

In [5]:
// From list of app packages, "appSample.txt" create json configs to run verifier
def createSamplesFromList():Unit = {
    val mode = Driver.SampleDeref
    val baseConfig = RunConfig(limit = 120, samples = 10)
    case class ApkSample(apk:String, pkg:String, outDir:File)
    val apkSamples = (expDir / "appSample.txt")
        .contentAsString
        .split("\n")map{v => 
            val pkg = v.split(":")(1)
            val apk = v.split(":")(0)
            val apkDirCfg = apk.toString.replace(fdroidBase.toString,"${baseDir}")
            val outDirCfg = "${baseDirOut}/" + pkg
            val outDir = expDir / pkg
            val getSample = outDir / "getSample.json"
            if(getSample.exists())getSample.delete()
            val cfg = baseConfig.copy(apkPath=apkDirCfg, outFolder=Some(outDirCfg))
            getSample.append(write[RunConfig](cfg))
            outDir.createIfNotExists(true)
            Action(Driver.SampleDeref, Some(expDir.toString), Some(fdroidBase.toString),
                   cfg)
        }
    apkSamples.foreach{action => 
        println(s"action: ${action}")
        Driver.runAction(action)
    }
//     apkSamples.foreach{println}
//     apkSamples.map{cfg => Driver.sampleDeref(cfg, cfg.apkPath, cfg.outFolder)}
}
//createSamplesFromList()

defined function createSamplesFromList

In [6]:
// count experiment json files
expDir.glob("**/sample.json").size

res5: Int = 14

In [7]:
// Run experiments
def runExperiments():Unit = {
    val experiments = Random.shuffle(expDir.glob("**/sample.json").toList)
    
    val experimentsPar = experiments.par
    val forkJoinPool = new java.util.concurrent.ForkJoinPool(4)
    experimentsPar.tasksupport = new ForkJoinTaskSupport(forkJoinPool)
    experimentsPar.foreach{cfg => 
    // val cfg = "/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_5/com.ridgelineapps.resdicegame/sample0.json"
        val runCFG = read[RunConfig](cfg.contentAsString)
        println(s"Running experiment: ${cfg.toString.split("/").takeRight(2).mkString("/")}")
        val runDir = File(runCFG.outFolder.get.replace("${baseDirOut}",expDir.toString))
        runDir.createIfNotExists(true)
        val stdoutF = runDir / "stdout.txt"
        val stderrF = runDir / "stderr.txt"
        val p = Process(s"java -jar ${path} -m verify -c $cfg -b ${fdroidBase.toString} -u ${expDir.toString}",
                        runDir.toJava,
                        "DYLD_LIBRARY_PATH"->"/Users/shawnmeier/software/z3/build")
        p ! ProcessLogger(v => stdoutF.append(v + "\n"),v => stderrF.append(v + "\n"))
        println(s"Running experiment: ${cfg.toString.split("/").takeRight(2).mkString("/")}")

    }
}
//runExperiments()

defined function runExperiments

In [8]:
// Extract witness files
def extractWitness():Unit = {
    val experiments = Random.shuffle(expDir.glob("**/sample*.json").toList)
    experiments.foreach{cfg =>
        val runCFG = read[RunConfig](cfg.contentAsString)
        val runDir = File(runCFG.outFolder.get.replace("${baseDirOut}",expDir.toString))
        println(s"${cfg}")
        val stdoutF = runDir / "stdoutextract.txt"
        val stderrF = runDir / "stderrextract.txt"
        val p = Process(s"java -jar ${path} -m readDB -c $cfg -u ${expDir.toString}",
                        runDir.toJava,
                        "DYLD_LIBRARY_PATH"->"/Users/shawnmeier/software/z3/build")
        p ! ProcessLogger(v => stdoutF.append(v + "\n"),v => stderrF.append(v + "\n"))
    }
}
extractWitness()

/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/net.khertan.forrunners/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/com.ridgelineapps.resdicegame/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/com.gpl.rpg.AndorsTrail/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/de.sigfood/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/org.vudroid/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/com.vadimfrolov.duorem/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/rino.org.tethercompanion/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/com.teamdc.stephendiniz.autoaway/sample.json
/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_15/org.bc_bd.mrwhite/sample.json
/Users/shawnmeier/Documents/source/bounder/ex

defined function extractWitness

In [11]:
// delete all results directories
def deleteResults():Unit = {
    val experiments = Random.shuffle(expDir.glob("**/sample.json").toList)
    experiments.map(v => v.toString.replace(".json","")).foreach{v => File(v).delete()}
}
//deleteResults()

defined function deleteResults

Count Results of Verification With No Model
-------------------------------------------

In [9]:
def interpResFile(resFile:File):String = {
    resFile.contentAsString.reverse.takeWhile(_!='}').reverse
}
def countResults(){
    val allResults = expDir.glob("**/result*.txt").toList
        .groupBy(interpResFile)
    val allResultsCount = expDir.glob("**/result*.txt").toList.groupBy(interpResFile).map{
        case (k,v) => 
            (k -> v.size)
    }
    allResults.foreach{case (k,v) => 
        println(k)
        println("-----------")
        println(s"count: ${v.size}")
        v.foreach{v => println(v.toString.split("/").takeRight(3).mkString("/"))}
    }
}
countResults()

Unreachable
-----------
count: 16
org.vudroid/sample/result_8.txt
org.vudroid/sample/result_7.txt
de.sigfood/sample/result_8.txt
de.sigfood/sample/result_5.txt
de.sigfood/sample/result_2.txt
com.gpl.rpg.AndorsTrail/sample/result_4.txt
com.teamdc.stephendiniz.autoaway/sample/result_3.txt
com.teamdc.stephendiniz.autoaway/sample/result_2.txt
itkach.aard2/sample/result_8.txt
itkach.aard2/sample/result_7.txt
itkach.aard2/sample/result_5.txt
com.serwylo.lexica/sample/result_2.txt
com.vadimfrolov.duorem/sample/result_8.txt
com.vadimfrolov.duorem/sample/result_5.txt
com.vadimfrolov.duorem/sample/result_3.txt
org.bc_bd.mrwhite/sample/result_8.txt
Witnessed
-----------
count: 2
com.teamdc.stephendiniz.autoaway/sample/result_1.txt
org.bc_bd.mrwhite/sample/result_0.txt
Proven
-----------
count: 58
org.vudroid/sample/result_9.txt
org.vudroid/sample/result_5.txt
org.vudroid/sample/result_4.txt
org.vudroid/sample/result_0.txt
org.vudroid/sample/result_3.txt
org.vudroid/sample/result_2.txt
de.sigfood/

defined function interpResFile
defined function countResults

Notes:
eclude org.ligi.etheremote - don't want to learn kotlin right now

- org.vudroid/
   - $\text{org.vudroid.core.DocumentView 135}$ - spec: map can only return null if gets null
   
- com.teamdc.stephendiniz.autoaway
app: https://f-droid.org/wiki/page/com.teamdc.stephendiniz.autoaway
file: Activity_Logger.java
Buffered reader can return null if the underlying file doesn’t have another line.  So the following code throws a null pointer exception if the file doesn’t have an even number of lines.  Admittedly, I think the developer already knew this because of the comment.


    551                                 //Should be in groups of TWO!
    552                                 while((line = bReader.readLine()) != null)
    553                                 {
    554                                         if (line.equals("\n"))
    555                                                 break;
    556
    557                                         Contact contactFromFile = new Contact(line, bReader.readLine());
    558                                         contacts.add(contactFromFile);
    559                                 }

- org.bc_bd.mrwhite
spec needed:
not I(ci a.onStart()) \/ NI(ci a.onStop(), ci a.onStart()) <= null:= ci a.getWindow




In [ ]:
// disjunction

Overall process

 - set of apps
     - set of all dereference
     - smallest spec to remove the most alarms
 - rq: The set of specs to prove a given location are "small"
     - spec: "small" means that few callbacks and callins are mentioned in the specification
     - search: we visit a small number of callbacks
     - trace abstraction: is small 
     - thresher hypothesis: materialized heap cells at any given location is small
 - 1. Number of dereferneces that we can prove with no fwk model - filter trival ones
 - 2. we develop a specifiction to address most of the alarms
 - plot cumulative distribution - x axis is number of specs, y total number of alarms
 - plot: average number of specs actually used by the verifier to prove a location
 - spec is not vacuous:
     - are callbacks reachable?
     - Is spec trivially unsound? (i.e. no traces are accepted)



Unreachable
-----------
org.ligi.etheremote/sample2/result.txt - kotlin library actually unreachable
org.ligi.etheremote/sample0/result.txt - library method
net.khertan.forrunners/sample1/result.txt - library method
Witnessed
-----------
net.khertan.forrunners/sample2/result.txt - (library) TODO: false alarm, handle literal read `r1 = ""; r1.toString`
de.sigfood/sample3/result.txt - TODO: witness on entering internal method
de.sigfood/sample0/result.txt - (library) TODO: false alarm, @this cannot be null.
Proven
-----------
org.ligi.etheremote/sample3/result.txt - "this.something"
net.khertan.forrunners/sample0/result.txt - "this.something"
de.sigfood/sample2/result.txt - (library) test if null before dereference
de.sigfood/sample1/result.txt - (library) test if null before dereference
com.ridgelineapps.resdicegame/sample0/result.txt - "this.something"
Timeout
-----------
org.vudroid/sample1/result.txt - probably proven with simple spec:
    - I(ci v.setOnItemClickListener(l)) <= cb l.onItemClick()
    - TODO: why don't we find the `<init>` where `files` field is null? 
com.ridgelineapps.resdicegame/sample1/result.txt
    - I(ci _.setPositiveButton(_,l)) <= cb l.onClick()
    - false <= l.onClick() - avoid these by saying "callback needs to be reachable"
    - "vacuous checks"
Error
-----------
org.ligi.etheremote/sample1/stderr.txt - soot unit graph bug - in lib
org.vudroid/sample0/stderr.txt - soot unit graph bug
org.vudroid/sample2/stderr.txt 
    - unsupported instruction `f0 cmpl 0.0F`
org.vudroid/sample3/stderr.txt 
    - unsupported instruction `f0 cmpl 0.0F`
net.khertan.forrunners/sample3/stderr.txt - soot unit graph bug
com.ridgelineapps.resdicegame/sample2/stderr.txt - soot unit graph bug
com.ridgelineapps.resdicegame/sample3/stderr.txt 
    - unsupported instruction `ArrayReference(r4,i4) := IntConst(1);`

Notes
-----
 - Need better definition of app/library code, most of the samples above are in libraries (all 3 unreachable)
 - Should exclude trivial things like `this.toString()` from sampling 
 - Fix a couple bugs
     - Read string literal should always be non-null
     - corner case where initial state is found when reaching the entry of an internal method
     - Missing instructions
     - Weird bug in soot unit graph
     
 - justify discarding apps
     - discard progard

Analysis of Errors
---------------------

In [ ]:
def printErrStats():Unit = {
    val exn = expDir.glob("**/stderr.txt").toList
        .groupBy(e => e.contentAsString.contains("Exception in"))
    val errors = exn.getOrElse(true,Set())
    val noErrors = exn.getOrElse(false,Set())
    println(s"no error: ${noErrors.size}")
    println(s"error: ${errors.size}")

    
    errors.foreach{e =>
        println(e.toString.split("/").takeRight(3).mkString("/"))
    }
    
    errors.foreach{e => 
        println()
        println(e)
        println("----")
        println(e.contentAsString)
    }
}
printErrStats()

Diagnose failures

In [ ]:
val cfg = File("/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_5/com.ridgelineapps.resdicegame/sample3.json")
val runCFG = read[RunConfig](cfg.contentAsString)
//<com.fasterxml.jackson.databind.deser.BuilderBasedDeserializer: java.lang.Object _deserializeUsingPropertyBased(com.fasterxml.jackson.core.JsonParser,com.fasterxml.jackson.databind.DeserializationContext)>
//Driver.readDB(runCFG)

Non termination analysis
-----------------------------

In [ ]:
import edu.colorado.plv.bounder.symbolicexecutor.state.DBOutputMode
def statesAtIf() = {
    val base = "/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_5/org.vudroid/sample2/"
    val db = DBOutputMode(s"${base}/paths.db")
    val stateMap = db.statesAtLoc("%isPre%true%")
    val f = File(base) / "statesAtLoc.txt"
    if(f.exists()) f.delete()
    val locWithMultipleStates = stateMap.filter{case (_,v) => v.size > 1}
        .toList.sortBy(v => v._2.size).reverse
        
    println(locWithMultipleStates.size)
    locWithMultipleStates.foreach{case (loc,stateset) =>
            f.append("\n")
            f.append(loc.toString)
            f.append("\n---------\n")
            stateset.foreach{state => f.append(state.toString + "\n")}
        }
}
statesAtIf()

Top n apps on fdroid
--------------------
https://fdroid.gitlab.io/metrics/f-droid.org/2021-02-22.json

In [ ]:
val stdout = new StringBuilder
val stderr = new StringBuilder
val p = Process(s"ls",
                File("/Users/shawnmeier/").toJava,
                "DYLD_LIBRARY_PATH"->"/Users/shawnmeier/software/z3/build")
p ! ProcessLogger(v => stdout.append(v + "\n"),v => stderr.append(v + "\n"))
println(s"stdout: ${stdout.toString}")
println(s"stderr: ${stderr.toString}")

Other issues to check out
------------------------------

https://github.com/AntennaPod/AntennaPod/issues/4914